<a href="https://colab.research.google.com/github/madhavadama/upxprojects/blob/master/Task1_sample.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<img src="https://shwetkm.github.io/upxlogo.png"></img>


# Live Project - Productionize Machine Learning models - Task 1(sample)

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
#Import basic packages

import warnings
warnings.filterwarnings("ignore")
import time
import pandas as pd               
import numpy as np
import pickle
import sys

import plotly
from plotly.data import iris
import sklearn
from sklearn.model_selection import train_test_split   #splitting data
from pylab import rcParams
from sklearn.linear_model import LinearRegression         #linear regression
from sklearn.metrics.regression import mean_squared_error #error metrics
from sklearn.metrics import mean_absolute_error
from sklearn.preprocessing import StandardScaler


In [0]:

sys.path.append('/content/drive/My Drive/python/')

In [0]:
#Import dictionaries for setting basic parameters and model parameters
from classifier_sc import classifier_config_dict,basic_params_dict, gridsearch_model_params_dict

In [5]:
classifier_config_dict

{'DecisionTreeClassifier': DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
                        max_features=None, max_leaf_nodes=None,
                        min_impurity_decrease=0.0, min_impurity_split=None,
                        min_samples_leaf=1, min_samples_split=2,
                        min_weight_fraction_leaf=0.0, presort=False,
                        random_state=None, splitter='best'),
 'LogisticRegression': LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                    intercept_scaling=1, l1_ratio=None, max_iter=100,
                    multi_class='warn', n_jobs=None, penalty='l2',
                    random_state=None, solver='warn', tol=0.0001, verbose=0,
                    warm_start=False),
 'SVM': SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
     decision_function_shape='ovr', degree=3, gamma='auto_deprecated',
     kernel='rbf', max_iter=-1, probability=False, random_state=None,
   

In [0]:
#Import Dataset
df_iris = iris()

In [0]:
basic_params_dict

{'X': ['sepal_length', 'sepal_width', 'petal_length', 'petal_width'],
 'seed': 123,
 'test_size': 0.3,
 'y': ['species_id']}

In [7]:
 gridsearch_model_params_dict

{'DecisionTreeClassifier': {'max_depth': range(1, 20, 2),
  'min_samples_split': range(10, 500, 20)},
 'LogisticRegression': {'C': [0.001, 0.009, 0.01, 0.09, 1, 5, 10, 25],
  'penalty': ['l1', 'l2']},
 'SVM': [{'C': [1, 10], 'kernel': ['linear']},
  {'C': [1, 10], 'gamma': [0.001, 0.0001], 'kernel': ['rbf']}]}

In [9]:
#View columns and paste in classifier_sc.py file with relevant columns
df_iris.describe()

,sepal_length,sepal_width,petal_length,petal_width,species_id
count,150.000000,150.000000,150.000000,150.000000,150.000000
mean,5.843333,3.054000,3.758667,1.198667,2.000000
std,0.828066,0.433594,1.764420,0.763161,0.819232
min,4.300000,2.000000,1.000000,0.100000,1.000000
25%,5.100000,2.800000,1.600000,0.300000,1.000000
50%,5.800000,3.000000,4.350000,1.300000,2.000000
75%,6.400000,3.300000,5.100000,1.800000,3.000000
max,7.900000,4.400000,6.900000,2.500000,3.000000


In [0]:
from sklearn.model_selection import GridSearchCV
# Create a class to perform base model operations
class BaseModelHelper:
    def __init__(self,base_param,base_model, base_model_param):
        self.base_param = base_param
        self.base_model = base_model
        #Initialize X
        self.X = self.base_param['X']
        #Initialize y
        self.y = self.base_param['y']
        #set random seed
        self.random_state = self.base_param['seed']
        #Set test_size
        self.test_size = self.base_param['test_size']
        #set base model params
        self.base_model_param = base_model_param
        self.keys = base_model.keys()
        self.grid_searches = {}

     
    #Function to standardize columns
    def normalize_columns(self):
        
        X=df_iris[self.X]
        self.y_norm=df_iris[self.y]

        #Scale the values
        scaler = StandardScaler()
        scaler.fit(X)

        # Scale and center the data
        fdf_normalized = scaler.transform(X)

        # Create a pandas DataFrame
        fdf_normalized = pd.DataFrame(data=fdf_normalized, index=X.index, columns=X.columns)
        
        #scaler.fit(y)

        # Scale and center the data
        #y_normalized = scaler.transform(y)

        # Create a pandas DataFrame
        #self.y = pd.DataFrame(data=y_normalized, index=y.index, columns=y.columns)
        #self.X=fdf_normalized
        return fdf_normalized
    def fit(self, X, y, cv=3, n_jobs=3, verbose=1, scoring=None, refit=False):
        for key in self.keys:
            print("Running GridSearchCV for %s." % key)
            model = self.base_model[key]
            params = self.base_model_param[key]
            gs = GridSearchCV(model, params, cv=cv, n_jobs=n_jobs,
                              verbose=verbose, scoring=scoring, refit=refit,
                              return_train_score=True)
            gs.fit(X,y)
            self.grid_searches[key] = gs    

    def grid_search(self):
      self.fit(self.X, self.y_norm)
    
    def score_summary(self, sort_by='mean_score'):
        def row(key, scores, params):
            d = {
                 'estimator': key,
                 'min_score': min(scores),
                 'max_score': max(scores),
                 'mean_score': np.mean(scores),
                 'std_score': np.std(scores),
            }
            return pd.Series({**params,**d})

        rows = []
        for k in self.grid_searches:
            print(k)
            params = self.grid_searches[k].cv_results_['params']
            scores = []
            for i in range(self.grid_searches[k].cv):
                key = "split{}_test_score".format(i)
                r = self.grid_searches[k].cv_results_[key]        
                scores.append(r.reshape(len(params),1))

            all_scores = np.hstack(scores)
            for p, s in zip(params,all_scores):
                rows.append((row(k, s, p)))

        df = pd.concat(rows, axis=1).T.sort_values([sort_by], ascending=False)

        columns = ['estimator', 'min_score', 'mean_score', 'max_score', 'std_score']
        columns = columns + [c for c in df.columns if c not in columns]

        return df[columns]

    #Function to perform train test split
    def train_test_split_base(self,X_norm):
        self.X=X_norm
        self.X_train, self.X_test, self.y_train, self.y_test = train_test_split(self.X,df_iris[self.y],random_state=self.random_state,test_size=self.test_size)
    
    #Building model
    def model_build(self):
        X_norm = self.normalize_columns()
        b.train_test_split_base(X_norm)
        for key in self.base_model:
            model=self.base_model[key]
            model.fit(self.X_train,self.y_train)
            y_pred_test = model.predict(self.X_test)
            return y_pred_test

In [0]:
# Calling BaseModelHelper with basic_params_dict and classifier_config_dict from classifier_sc.py 
b=BaseModelHelper(basic_params_dict,classifier_config_dict, gridsearch_model_params_dict)
b.normalize_columns()
# Model Building
y_pred_test=b.model_build()

In [37]:
#Output predictions
y_pred_test

array([2, 3, 3, 2, 1, 2, 2, 1, 1, 2, 3, 1, 2, 3, 3, 3, 1, 1, 2, 1, 1, 2,
       1, 3, 1, 1, 1, 3, 3, 1, 3, 2, 1, 1, 2, 2, 3, 1, 1, 2, 2, 1, 3, 3,
       3])

In [38]:
b.grid_search()

Running GridSearchCV for DecisionTreeClassifier.
Fitting 3 folds for each of 250 candidates, totalling 750 fits


[Parallel(n_jobs=3)]: Using backend LokyBackend with 3 concurrent workers.
[Parallel(n_jobs=3)]: Done 124 tasks      | elapsed:    3.4s
[Parallel(n_jobs=3)]: Done 750 out of 750 | elapsed:    6.0s finished
/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_search.py:814: DeprecationWarning:

The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.



Running GridSearchCV for LogisticRegression.
Fitting 3 folds for each of 16 candidates, totalling 48 fits


[Parallel(n_jobs=3)]: Using backend LokyBackend with 3 concurrent workers.
[Parallel(n_jobs=3)]: Done  43 out of  48 | elapsed:    2.8s remaining:    0.3s
[Parallel(n_jobs=3)]: Done  48 out of  48 | elapsed:    2.9s finished
/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_search.py:814: DeprecationWarning:

The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.



Running GridSearchCV for SVM.
Fitting 3 folds for each of 6 candidates, totalling 18 fits


[Parallel(n_jobs=3)]: Using backend LokyBackend with 3 concurrent workers.
[Parallel(n_jobs=3)]: Done  18 out of  18 | elapsed:    2.8s finished
/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_search.py:814: DeprecationWarning:

The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.



In [39]:
b.score_summary()


DecisionTreeClassifier
LogisticRegression
SVM


,estimator,min_score,mean_score,max_score,std_score,C,gamma,kernel,max_depth,min_samples_split,penalty
267,SVM,0.921569,0.973856,1,0.0369729,10,NaN,linear,NaN,NaN,NaN
266,SVM,0.921569,0.973856,1,0.0369729,1,NaN,linear,NaN,NaN,NaN
264,LogisticRegression,0.921569,0.96732,1,0.0333269,25,NaN,NaN,NaN,NaN,l1
262,LogisticRegression,0.921569,0.96732,1,0.0333269,10,NaN,NaN,NaN,NaN,l1
260,LogisticRegression,0.901961,0.960784,1,0.0423578,5,NaN,NaN,NaN,NaN,l1
...,...,...,...,...,...,...,...,...,...,...,...
43,DecisionTreeClassifier,0.333333,0.333333,0.333333,0,NaN,NaN,NaN,3,370,NaN
137,DecisionTreeClassifier,0.333333,0.333333,0.333333,0,NaN,NaN,NaN,11,250,NaN
138,DecisionTreeClassifier,0.333333,0.333333,0.333333,0,NaN,NaN,NaN,11,270,NaN
139,DecisionTreeClassifier,0.333333,0.333333,0.333333,0,NaN,NaN,NaN,11,290,NaN
